# EDA Stations and Paths

In [24]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import date, time, datetime

In [25]:
#Create dataframe from CSV
df = pd.read_csv("../data/fromAPI/cleaned_hourly_all.csv")

# EDA delay per path

In [26]:
# #Total number of delays 
# total_delays = df[df["arrival_delay_check"] == "delay"].shape[0]

# #Path delays
# path_delay_counts = df[df["arrival_delay_check"] == "delay"].groupby("path").size().reset_index(name='count')


# #Normalized counts
# path_delay_counts['normalized_count'] = path_delay_counts['count'] / total_delays

In [27]:
#arrival delays occurances as percentage of total trains that have taken that path
df2 = df.groupby("path", as_index=False).arrival_delay_check.count()
df3 = df[df["arrival_delay_check"] == "delay"].groupby("path", as_index=False).arrival_delay_check.count()
paths = pd.merge(df2, df3, how="left", on="path", suffixes=("", "_delay"))
paths["delay%"] = (paths["arrival_delay_check_delay"] / paths["arrival_delay_check"])*100

#arrival delays occurances as percentage of total amount of delays in dataset
paths2 = df3
paths2["delay_from_total%"] = (paths2["arrival_delay_check"] / paths2["arrival_delay_check"].sum())*100

In [28]:
#only arrival delays percentage occurences after first stop
paths.sort_values("delay%", inplace=True, ascending=False)
paths.reset_index(inplace=True)
paths[~paths["path"].str.contains(r'\|')].head(100) #excluding strings with the pillar symbol


,index,path,arrival_delay_check,arrival_delay_check_delay,delay%
457,25863,Tangerhütte,1,1.0,100.000000
830,27163,Weißenfels,2,2.0,100.000000
2433,7762,Esslingen(Neckar),10,6.0,60.000000
2512,5631,Dillenburg,40,23.0,57.500000
2725,20746,Neufahrn(b Freising),6,3.0,50.000000
...,...,...,...,...,...
12561,3251,Berlin-Charlottenburg,99,6.0,6.060606
12564,14942,Kassel-Wilhelmshöhe,99,6.0,6.060606
12581,1388,Bamberg,366,22.0,6.010929
12582,19293,Mönchengladbach Hbf,333,20.0,6.006006


In [29]:
paths2.sort_values("delay_from_total%", inplace=True, ascending=False)
paths2.reset_index(inplace=True)
paths2[~paths2["path"].str.contains(r'\|')].head(100)

,index,path,arrival_delay_check,delay_from_total%
0,11068,München Hbf,168,0.158998
1,4516,Frankfurt(Main)Hbf,126,0.119248
3,6251,Hamburg Hbf,94,0.088963
7,3104,Dortmund Hbf,86,0.081392
13,14447,Stuttgart Hbf,79,0.074767
...,...,...,...,...
4423,13879,Schorndorf,7,0.006625
4523,2967,Dietzenbach Bahnhof,7,0.006625
4547,8550,Kassel Hbf,7,0.006625
4552,14182,Singen(Hohentwiel),7,0.006625


In [30]:
# # Sort the normalized counts to get the top 10 paths
# top_10_normalized_counts = path_delay_counts.sort_values(by='normalized_count', ascending=False).head(10)

# #Percentage
# top_10_normalized_counts["percentage"] = top_10_normalized_counts["normalized_count"] * 100

In [31]:
#showing the 10 paths with the biggest percentage of delays
# Create a bar chart using Plotly
fig = px.bar(
    paths[~paths["path"].str.contains(r'\|')].head(10),
    x='path',
    y='delay%',
    title='Top 10 Paths with Highest Proportional delay amount',
    labels={'path': 'Path', 'normalized_count': 'Proportion of Delays (%)'}
    
)
#Update
#fig.update_xaxes(tickangle=90)
# Hide X-axis labels by setting tickvals and ticktext to empty lists
#fig.update_xaxes(tickvals=[], ticktext=[])
fig.update_layout(
    height=600,
    margin=dict(l=20, r=20, t=50, b=100)
)


#actually show y in %%!
fig.update_yaxes(tickformat=".2f")

fig.show()

In [32]:
#showing the 10 paths with the biggest percentage of delays
# Create a bar chart using Plotly
fig = px.bar(
    paths2[~paths2["path"].str.contains(r'\|')].head(10),
    x='path',
    y='delay_from_total%',
    title='Top 10 Paths with Highest Proportional delay amount',
    labels={'path': 'Path', 'normalized_count': 'Proportion of Delays (%)'}
    
)
#Update
fig.update_layout(
    height=600,
    margin=dict(l=20, r=20, t=50, b=100)
)


#actually show y in %%!
fig.update_yaxes(tickformat=".2f")

fig.show()

# EDA delay per Station

In [33]:
df

,ID,train,path,eva_nr,category,name,state,city,zip,long,lat,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,depature_delay_m,hour,arrival_delay_check,departure_delay_check
0,4083473720689476753-2406231250-4,20,Stolberg(Rheinl)Hbf|Eilendorf|Aachen-Rothe Erde,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.767800,2024-06-23 13:01:00,2024-06-23 13:02:00,2024-06-23 13:03:00,2024-06-23 13:04:00,2,2,0,on_time,on_time
1,-8892239479013746475-2406231212-14,20,Stolberg(Rheinl)Hbf Gl.44|Eschweiler-St.Jöris|...,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.767800,2024-06-23 13:00:00,2024-06-23 13:01:00,2024-06-23 13:05:00,2024-06-23 13:06:00,5,5,0,on_time,on_time
2,3772983581833112391-2406231337-1,33,NaN,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.767800,NaN,2024-06-23 13:37:00,NaN,NaN,0,0,0,on_time,on_time
3,-7760689325071259199-2406231337-1,29,NaN,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.767800,NaN,2024-06-23 13:37:00,NaN,NaN,0,0,0,on_time,on_time
4,3762663024269879642-2406231318-1,9,NaN,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.767800,NaN,2024-06-23 13:18:00,NaN,2024-06-23 13:18:00,0,0,0,on_time,on_time
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1370584,-2524967729471299033-2406271333-36,NaN,Bergisch Gladbach|Duckterath|Köln-Dellbrück|Kö...,8001580,4,Düsseldorf Flughafen Terminal,Nordrhein-Westfalen,Düsseldorf,40474,6.766979,51.278517,2024-06-27 15:15:00,NaN,2024-06-27 15:16:00,NaN,1,0,98,on_time,on_time
1370585,7773222948651306726-2406271544-1,11,NaN,8001580,4,Düsseldorf Flughafen Terminal,Nordrhein-Westfalen,Düsseldorf,40474,6.766979,51.278517,NaN,2024-06-27 15:44:00,NaN,NaN,0,0,98,on_time,on_time
1370586,5484088617416038144-2406271353-36,NaN,Bergisch Gladbach|Duckterath|Köln-Dellbrück|Kö...,8001580,4,Düsseldorf Flughafen Terminal,Nordrhein-Westfalen,Düsseldorf,40474,6.766979,51.278517,2024-06-27 15:35:00,NaN,2024-06-27 15:36:00,NaN,1,0,98,on_time,on_time
1370587,-852858026441192630-2406271524-1,11,NaN,8001580,4,Düsseldorf Flughafen Terminal,Nordrhein-Westfalen,Düsseldorf,40474,6.766979,51.278517,NaN,2024-06-27 15:24:00,NaN,2024-06-27 15:24:00,0,0,98,on_time,on_time


In [41]:
#arrival delays per train for Stations
df_station = df.groupby("name", as_index=False).arrival_delay_check.count()
df_station2 = df[df["arrival_delay_check"] == "delay"].groupby("name", as_index=False).arrival_delay_check.count()
station = pd.merge(df_station, df_station2, how="left", on="name", suffixes=("", "_delay"))
station["delay%"] = (station["arrival_delay_check_delay"] / station["arrival_delay_check"])*100

#arrival delays occurances as percentage of total amount of delays in dataset
station2 = df_station2
station2["delay_from_total%"] = (station2["arrival_delay_check"] / station2["arrival_delay_check"].sum())*100

In [42]:
station.sort_values("delay%", inplace=True, ascending=False)
station.reset_index(inplace=True)
station.head(10)

,index,name,arrival_delay_check,arrival_delay_check_delay,delay%
0,595,Frankfurt am Main Flughafen Fernbahnhof,933,443.0,47.481243
1,1727,St. Goar,188,76.0,40.425532
2,1120,Limburg Süd,83,33.0,39.759036
3,1898,Weißenthurm,299,114.0,38.127090
4,1264,Mülheim-Kärlich,315,118.0,37.460317
5,249,Boppard-Bad Salzig,181,66.0,36.464088
6,285,Brohl,172,62.0,36.046512
7,73,Bad Breisig,311,110.0,35.369775
8,1588,Roisdorf,208,71.0,34.134615
9,1430,Oberwinter,342,110.0,32.163743


In [43]:
station2["arrival_delay_check"]

0       177
1        99
2        90
3        70
4        77
       ... 
1968     16
1969     24
1970     17
1971    114
1972     29
Name: arrival_delay_check, Length: 1973, dtype: int64

- Top 10 Bahnhöfe mit anzahl verspätungen geteilt durch anzahl fahrten (also quasi Prozentsatz der verspätungen am Bahnhof)(balkendiagramm)<br>


In [44]:
#showing the 10 paths with the biggest percentage of delays
# Create a bar chart using Plotly
fig = px.bar(
    station.head(10),
    x='name',
    y='delay%',
    title='Top 10 Stations with Highest Proportional delay amount',    
)
#Update
fig.update_layout(
    height=600,
    margin=dict(l=20, r=20, t=50, b=100)
)


#actually show y in %%!
fig.update_yaxes(tickformat=".2f")

fig.show()

In [45]:
station2.sort_values("delay_from_total%", inplace=True, ascending=False)
station2.reset_index(inplace=True)
station2.head(10)

,index,name,arrival_delay_check,delay_from_total%
0,992,Köln Hbf,1004,0.950200
1,416,Düsseldorf Hbf,872,0.825273
2,570,Frankfurt (Main) Hbf,782,0.740096
3,403,Duisburg Hbf,724,0.685204
4,763,Hannover Hbf,676,0.639776
5,368,Dortmund Hbf,666,0.630312
6,517,Essen Hbf,645,0.610437
7,1745,Stuttgart Hbf,640,0.605705
8,1168,Mannheim Hbf,601,0.568795
9,1379,Nürnberg Hbf,597,0.565009


In [46]:
#showing the 10 paths with the biggest percentage of delays
# Create a bar chart using Plotly
fig = px.bar(
    station2.head(10),
    x='name',
    y='delay_from_total%',
    title='Top 10 Stations with Highest Proportional delay amount',    
)
#Update
fig.update_layout(
    height=600,
    margin=dict(l=20, r=20, t=50, b=100)
)


#actually show y in %%!
fig.update_yaxes(tickformat=".2f")

fig.show()